Compose function - Covulational to Images 
f(Img) → bird

I used Cifar10 lib

In [ ]:
import torch
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.datasets import CIFAR10

In [ ]:
BASE_DIR = '/content/drive/MyDrive/datasets'
DATA_DIR = '/content/drive/MyDrive/datasets/cifar10'
CATEGORIES = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
def get_path(relpath):
  return os.path.join(BASE_DIR, relpath)

In [ ]:
cifar10_train = CIFAR10(DATA_DIR, train=True, download=True)
cifar10_test = CIFAR10(DATA_DIR, train=False, download=True)

In [ ]:
(len(cifar10_train), len(cifar10_test))

Expected result: 

*(50000, 10000)*

In [ ]:
def lookat_dataset(dataset, istensor=False):
  figure = plt.figure(figsize=(8, 8))
  rows, cols = 2, 2
  for i in range(1, 5):
      sample_idx = torch.randint(len(dataset), size=(1,)).item()
      img, label = dataset[sample_idx]
      figure.add_subplot(rows, cols, i)
      plt.title(CATEGORIES[label])
      plt.axis("off")
      if istensor:
        plt.imshow(img.squeeze().permute(1, 2, 0))
      else:
        plt.imshow(img)
  plt.show()

In [ ]:
lookat_dataset(cifar10_train)

Expected Result:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/aaeffd09-bbf2-47cf-a464-28a981f96abc/Untitled.png)

In [ ]:
prep_transform = T.Compose([
                    T.ToTensor()
                  ])

# Applying a transform
tensor_train = CIFAR10(DATA_DIR, train=True, download=False,
                         transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False,
                         transform=prep_transform)

In [ ]:
# We need identfy the labels to normalize
# Normalizing data
imgs = torch.stack([img_t for img_t, _ in tensor_train], dim=3)
imgs.shape

In [ ]:
imgs.view(3, -1).mean(dim=1)

In [ ]:
imgs.view(3, -1).std(dim=1)

In [ ]:
prep_transform = T.Compose([
                    T.ToTensor(),
										#O normalize added with numbers found.
                    T.Normalize(
                        (0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)
                    )
                  ])

# Applying a transform
tensor_train = CIFAR10(DATA_DIR, train=True, download=False,
                         transform=prep_transform)
tensor_test = CIFAR10(DATA_DIR, train=False, download=False,
                         transform=prep_transform)

In [ ]:
lookat_dataset(tensor_train, True)

Expected results: 
![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/463b78f1-eeec-4846-8865-e95347290b84/Untitled.png)

In [ ]:
#Create dataloaders
batch_size = 64
train_loader = DataLoader(tensor_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(tensor_test, batch_size=batch_size, shuffle=False)

In [ ]:
from torch import nn

In [ ]:
class MLPClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    
    self.layers = nn.Sequential(
				#3=RGB, 32*32=image size, break blocks
        nn.Linear(3 * 32*32, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10),
    )

  def forward(self, x):
    v = self.flatten(x)
    return self.layers(v)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Run on {device}")

In [ ]:
model = MLPClassifier().to(device)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(model, dataloader, loss_func, optimizer):
  model.train()
  cumloss = 0.0

  for imgs, labels in dataloader:
    imgs, labels = imgs.to(device), labels.to(device)
    
    optimizer.zero_grad()

    pred = model(imgs)

    loss = loss_func(pred, labels)
    loss.backward()
    optimizer.step()

    cumloss += loss.item()

  return cumloss / len(dataloader)

def validate(model, dataloader, loss_func):
  model.eval()
  cumloss = 0.0

  with torch.no_grad():
    for imgs, labels in dataloader:
      imgs, labels = imgs.to(device), labels.to(device)

      pred = model(imgs)
      loss = loss_func(pred, labels)
      cumloss += loss.item()

  return cumloss / len(dataloader)

In [ ]:
def plot_losses(losses):
  fig = plt.figure(figsize=(13, 5))
  ax = fig.gca()
  for loss_name, loss_values in losses.items():  
    ax.plot(loss_values, label=loss_name)
  ax.legend(fontsize="16")
  ax.set_xlabel("Iteration", fontsize="16")
  ax.set_ylabel("Loss", fontsize="16")
  ax.set_title("Loss vs iterations", fontsize="16")

In [ ]:
#See data loss on train
epochs = 41
train_losses = []
test_losses = []
for t in range(epochs):
  train_loss = train(model, train_loader, loss_func, optimizer)
  train_losses.append(train_loss)
  if t % 10 == 0:
    print(f"Epoch: {t}; Train Loss: {train_loss}")
  
  test_loss = validate(model, test_loader, loss_func)
  test_losses.append(test_loss)

In [ ]:
losses = {"Train loss": train_losses, "Test loss": test_losses}
plot_losses(losses)

Expected result:
![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/d001451f-1b73-4565-8976-22d07bc2ff5b/Untitled.png)

In [ ]:
#To evaluate results

def make_confusion_matrix(model, loader, n_classes):
  confusion_matrix = torch.zeros(n_classes, n_classes, dtype=torch.int64)
  with torch.no_grad():
    for i, (imgs, labels) in enumerate(loader):
      imgs = imgs.to(device)
      labels = labels.to(device)
      outputs = model(imgs)
      _, predicted = torch.max(outputs, 1)
      for t, p in zip(torch.as_tensor(labels, dtype=torch.int64).view(-1), 
                      torch.as_tensor(predicted, dtype=torch.int64).view(-1)):
        confusion_matrix[t, p] += 1
  return confusion_matrix

def evaluate_accuracy(model, dataloader, classes, verbose=True):
  # prepare to count predictions for each class
  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}

  confusion_matrix = make_confusion_matrix(model, dataloader, len(classes))
  if verbose:
    total_correct = 0.0
    total_prediction = 0.0
    for i, classname in enumerate(classes):
      correct_count = confusion_matrix[i][i].item()
      class_pred = torch.sum(confusion_matrix[i]).item()

      total_correct += correct_count
      total_prediction += class_pred

      accuracy = 100 * float(correct_count) / class_pred
      print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                    accuracy))
  print("Global acccuracy is {:.1f}".format(100 * total_correct/total_prediction))
  return confusion_matrix

def test(model, dataloader, classes):
  # prepare to count predictions for each class
  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}

  # again no gradients needed
  with torch.no_grad():
      for images, labels in dataloader:
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          _, predictions = torch.max(outputs, 1)
          # collect the correct predictions for each class
          for label, prediction in zip(labels, predictions):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1

  # print accuracy for each class
  total_correct = 0.0
  total_prediction = 0.0
  for classname, correct_count in correct_pred.items():
      total_correct += correct_count
      total_prediction += total_pred[classname]
      accuracy = 100 * float(correct_count) / total_pred[classname]
      print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                    accuracy))
  print("Global acccuracy is {:.1f}".format(100 * total_correct/total_prediction))

In [ ]:
confusion_matrix = evaluate_accuracy(model, test_loader, CATEGORIES)

In [ ]:
import seaborn as sn

In [ ]:
plt.figure(figsize=(12, 12))
sn.set(font_scale=1.4)
sn.heatmap(confusion_matrix.tolist(), 
           annot=True, annot_kws={"size": 16}, fmt='d')

Expeted Results:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/eab8925b-84e9-4452-af4e-1e92b175f22d/Untitled.png)

In [ ]:
#inite covolutional network

class ConvolutionalModel(nn.Module):
  def __init__(self):
      super().__init__()
			#layer covolutional 2D
      self.convlayers = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=(3, 3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

        nn.Conv2d(16, 32, kernel_size=(3, 3)),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),

      )
			#classify
      self.linearlayers = nn.Sequential(
					#simulate on CPU to see entry valor, or calculate with border difference.
          nn.Linear(1152, 256),
          nn.ReLU(),
          nn.Linear(256, 10)
      )

  def forward(self, x):
      x = self.convlayers(x)
      x = torch.flatten(x, 1)
      return self.linearlayers(x)

In [ ]:
convmodel = ConvolutionalModel().to(device)

In [ ]:
loss_func2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.SGD(convmodel.parameters(), lr=0.001)

In [ ]:
epochs = 51
conv_train_losses = []
conv_test_losses = []
for t in range(epochs):
  train_loss = train(convmodel, train_loader, loss_func2, optimizer2)
  conv_train_losses.append(train_loss)
  if t % 10 == 0:
    print(f"Epoch: {t}; Train Loss: {train_loss}")
  test_loss = validate(convmodel, test_loader, loss_func2)
  conv_test_losses.append(test_loss)

In [ ]:
conv_losses = {"Train Loss": conv_train_losses, "Test Loss": conv_test_losses}
plot_losses(conv_losses)

Expected result:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/a4d77a8a-4fff-4f56-b662-7fee839c1666/Untitled.png)

In [ ]:
conv_confusion_matrix = evaluate_accuracy(convmodel, test_loader, CATEGORIES)

In [ ]:
plt.figure(figsize=(12, 12))
sn.set(font_scale=1.4)
sn.heatmap(confusion_matrix.tolist(), 
           annot=True, annot_kws={"size": 16}, fmt='d')

Expected result:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/1615e07d-2d98-45ec-8961-1bb382a5a7a0/Untitled.png)

In [ ]:
from PIL import Image

In [ ]:
#pass your image to test here
img = Image.open(get_path('/content/drive/MyDrive/angry.JPG'))
img

Expected result:

See your image 

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/8bbb0647-faef-4ce8-ad0c-f5ab6c24087f/Untitled.png)

In [ ]:
prep_transforms = T.Compose(
    [T.Resize((32, 32)),
     T.ToTensor(),
     T.Normalize( (0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616) )
     ]
)
img_tensor = prep_transforms(img)

In [ ]:
plt.imshow(img_tensor.permute(1,2, 0))

Expected result:

See your image transformed
![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/b7435336-76da-4547-874f-27f78247535d/Untitled.png)

In [ ]:
batch = img_tensor.unsqueeze(0).to(device)

In [ ]:
convmodel.eval()

Expected result:
See your covolutional model

ConvolutionalModel(
(convlayers): Sequential(
(0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
(1): ReLU()
(2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
(3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
(4): ReLU()
(5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
(linearlayers): Sequential(
(0): Linear(in_features=1152, out_features=256, bias=True)
(1): ReLU()
(2): Linear(in_features=256, out_features=10, bias=True)
))

In [ ]:
output = convmodel(batch)
output

In [ ]:
logits = torch.nn.functional.softmax(output, dim=1) * 100
prob_dict = {}
for i, classname in enumerate(CATEGORIES):
  prob = logits[0][i].item()
  print(f"{classname} score: {prob:.2f}")
  prob_dict[classname] = [prob]

In [ ]:
import pandas as pd

In [ ]:
df_prob = pd.DataFrame.from_dict(prob_dict)

In [ ]:
df_prob.plot(kind='barh', figsize=(12, 8))

Expected result:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/6c527c41-12fa-4055-b6a8-f7748b496682/Untitled.png)

In [ ]:
# Save your models

torch.save(model.state_dict(), '/content/drive/MyDrive/mlp_model_weights.pth')

torch.save(convmodel.state_dict(), '/content/drive/MyDrive/conv_model_weights.pth')